In [ ]:
from __future__ import annotations

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

import qubex.pulse as qp
from qubex.simulator import (
    Control,
    Coupling,
    QuantumSimulator,
    QuantumSystem,
    SimulationResult,
    Transmon,
)

qp.set_sampling_period(Control.SAMPLING_PERIOD)

In [ ]:
qubits = [
    Transmon(
        label="Q0",
        dimension=3,
        frequency=7.0,
        anharmonicity=-0.333,
        # relaxation_rate=0.00005,
        # dephasing_rate=0.00005,
    ),
    Transmon(
        label="Q1",
        dimension=3,
        frequency=8.0,
        anharmonicity=-0.333,
        # relaxation_rate=0.00005,
        # dephasing_rate=0.00005,
    ),
]

couplings = [
    Coupling(pair=("Q0", "Q1"), strength=0.01),
]

system = QuantumSystem(
    objects=qubits,
    couplings=couplings,
)

simulator = QuantumSimulator(system)

In [ ]:
control_qubit = qubits[0]
target_qubit = qubits[1]

control_label = control_qubit.label
target_label = target_qubit.label

In [ ]:
def cr_drive(
    duration: float,
    amplitude: float,
    ramp: float,
    phase: float,
) -> Control:
    return Control(
        target=control_label,
        frequency=target_qubit.frequency,
        waveform=qp.FlatTop(
            duration=duration,
            amplitude=2 * np.pi * amplitude,
            tau=ramp,
            phase_shift=phase,
        ),
    )


cr_drive(
    duration=100,
    amplitude=0.1,
    ramp=30,
    phase=0,
).plot()

In [ ]:
def cancel_drive(
    duration: float,
    amplitude: float,
    ramp: float,
    phase: float,
) -> Control:
    return Control(
        target=target_label,
        frequency=target_qubit.frequency,
        waveform=qp.FlatTop(
            duration=duration,
            amplitude=2 * np.pi * amplitude,
            tau=ramp,
            phase_shift=phase,
        ),
    )


cancel_drive(
    duration=100,
    amplitude=0.01,
    ramp=30,
    phase=0,
).plot()

In [ ]:
def simulate_cr(
    cr_duration: int,
    cr_amplitude: float,
    cr_ramp: float,
    cr_phase: float,
    cancel_amplitude: float | None = None,
    cancel_phase: float | None = None,
    control_state: str = "0",
    plot: bool = False,
    n_samples: int = 256,
) -> SimulationResult:
    controls = [cr_drive(cr_duration, cr_amplitude, cr_ramp, cr_phase)]
    if cancel_amplitude is not None and cancel_phase is not None:
        controls.append(cancel_drive(cr_duration, cancel_amplitude, cr_ramp, cr_phase))
    initial_state = system.state(
        {
            control_label: control_state,
            target_label: "0",
        },
    )
    result = simulator.mesolve(
        controls=controls,
        initial_state=initial_state,
    )
    if plot:
        result.plot_bloch_vectors(target_label, n_samples=n_samples)
        result.display_bloch_sphere(target_label, n_samples=n_samples)
    return result

In [ ]:
cr_duration = 1000
cr_amplitude = 0.2
cr_ramp = 30
cr_phase = 0.0

In [ ]:
result_0 = simulate_cr(
    cr_duration=cr_duration,
    cr_amplitude=cr_amplitude,
    cr_ramp=cr_ramp,
    cr_phase=cr_phase,
    control_state="0",
    plot=True,
)

In [ ]:
result_1 = simulate_cr(
    cr_duration=cr_duration,
    cr_amplitude=cr_amplitude,
    cr_ramp=cr_ramp,
    cr_phase=cr_phase,
    control_state="1",
    plot=True,
)

In [ ]:
n_samples = 100
times = result_0.get_times(n_samples=n_samples)
vectors_0 = result_0.get_bloch_vectors(target_label, n_samples=n_samples)
vectors_1 = result_1.get_bloch_vectors(target_label, n_samples=n_samples)

In [ ]:
R = np.sqrt(np.sum((vectors_0 + vectors_1) ** 2, axis=1))

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=4,
    cols=1,
    shared_xaxes=True,
)

for i, component in enumerate(["X", "Y", "Z"]):
    fig.add_trace(
        go.Scatter(
            mode="lines+markers",
            x=times,
            y=vectors_0[:, i],
            name=f"0_{component}",
        ),
        row=i + 1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            mode="lines+markers",
            x=times,
            y=vectors_1[:, i],
            name=f"1_{component}",
        ),
        row=i + 1,
        col=1,
    )

fig.add_trace(
    go.Scatter(
        mode="lines+markers",
        x=times,
        y=R,
        name="R",
    ),
    row=4,
    col=1,
)

fig.update_layout(
    title_text="Bloch vectors",
    showlegend=False,
    height=500,
    yaxis_range=[-1, 1],
)

fig.update_xaxes(title_text="Time (ns)", row=4, col=1)

fig.update_yaxes(title_text="〈X〉", row=1, col=1)
fig.update_yaxes(title_text="〈Y〉", row=2, col=1)
fig.update_yaxes(title_text="〈Z〉", row=3, col=1)
fig.update_yaxes(title_text="|R|", row=4, col=1)

fig.show()

In [ ]:
indices = (times >= cr_ramp) & (times < times[-1] - cr_ramp)
times_ = times[indices] - cr_ramp * 0.5
vectors_0_ = vectors_0[indices]
vectors_1_ = vectors_1[indices]

In [ ]:
import qctrlvisualizer as qv

from qubex.analysis.fitting import fit_rotation3d

Omega_0 = fit_rotation3d(times_, vectors_0_)
qv.display_bloch_sphere_from_bloch_vectors(vectors_0_)

Omega_1 = fit_rotation3d(times_, vectors_1_)
qv.display_bloch_sphere_from_bloch_vectors(vectors_1_)

In [ ]:
Omega = np.concatenate(
    [
        0.5 * (Omega_0 + Omega_1),
        0.5 * (Omega_0 - Omega_1),
    ]
)
coeffs = dict(
    zip(
        ["IX", "IY", "IZ", "ZX", "ZY", "ZZ"],
        0.5 * Omega / (2 * np.pi),
    )
)

In [ ]:
coeffs

In [ ]:
for key, value in coeffs.items():
    print(f"{key}: {value * 1e3:+.6f} MHz")

In [ ]:
phi_est = np.arctan2(coeffs["ZY"], coeffs["ZX"])
phi_est

In [ ]:
cancel_amplitude_est = -coeffs["IX"] * 2
cancel_amplitude_est